## Ottengo i dati necessari
Poiché non esiste alcun dataset 

In [83]:
from rdflib import Graph
from owlready2 import get_ontology

# Percorso al file dell'ontologia in formato Turtle
ontology_path = "data/Ontology/mtg_ontology_updated.owl.tll"

# Crea un grafo RDF e carica il file Turtle usando rdflib
g = Graph()
g.parse(ontology_path, format="turtle")

# Salva il grafo RDF in un formato supportato da owlready2 (come RDF/XML)
converted_ontology_path = "converted_mtg_ontology.owl"
g.serialize(destination=converted_ontology_path, format="xml")

# Carica l'ontologia convertita usando owlready2
onto = get_ontology(converted_ontology_path).load()

# Crea un dizionario per raccogliere le proprietà per ogni classe
class_hierarchy = {}

# Popola il dizionario con le sottoclassi
for cls in onto.classes():
    class_hierarchy[cls] = {
        'subclasses': list(cls.subclasses()),
        'properties': [prop for prop in cls.is_a if hasattr(prop, 'name') and prop.name != "Thing"]
    }

# stampa ricorsivamente le sottoclassi
def print_hierarchy(cls, indent=""):
    subclasses = class_hierarchy[cls]['subclasses']
    for i, subclass in enumerate(subclasses):
        connector = "├" if i < len(subclasses) - 1 else "└"
        print(f"{indent} {connector} {subclass.name}")
        print_hierarchy(subclass, indent + "  ")


In [84]:

# Stampa l'albero a partire dalla classe Thing
print("Albero dell'ontologia:")
thing_class = onto.Thing  # Supponiamo che Thing sia la radice dell'ontologia
print(thing_class.name)
print_hierarchy(thing_class)

Albero dell'ontologia:
Thing


In [39]:
# Stampa l'albero a partire dalla classe NamedThing
print("Albero dell'ontologia:")
thing_class = onto.NamedThing 
print(thing_class.name)
print_hierarchy(thing_class)

Albero dell'ontologia:
NamedThing
 ├ Card
   ├ Artifact
   ├ Creature
   ├ Sorcery
   ├ Enchantment
   ├ Instant
   └ Land
 └ Permanent
   ├ AnyArtifact
     ├ Artifact
     └ ArtifactToken
   ├ AnyCreature
     ├ Creature
     └ CreatureToken
   ├ Token
     ├ CreatureToken
     ├ ArtifactToken
     └ EnchantmentToken
   ├ AnyEnchantment
     ├ Enchantment
     └ EnchantmentToken
   └ Land


In [85]:
import owlready2
# Percorso al file dell'ontologia in formato Turtle
ontology_path = "data/Ontology/mtg_ontology.owl.tll"

# Crea un grafo RDF e carica il file Turtle usando rdflib
g = Graph()
g.parse(ontology_path, format="turtle")

# Salva il grafo RDF in un formato supportato da owlready2 (come RDF/XML)
converted_ontology_path = "converted_mtg_ontology.owl"
g.serialize(destination=converted_ontology_path, format="xml")

# Carica l'ontologia convertita usando owlready2
onto = get_ontology(converted_ontology_path).load()

In [91]:

from prompt_toolkit import prompt
from prompt_toolkit.auto_suggest import AutoSuggestFromHistory
from prompt_toolkit.completion import Completer, Completion
# Definisci un completatore personalizzato
class OntologyCompleter(Completer):
    def __init__(self, ontology):
        self.ontology = ontology
        self.classes = [cls.name for cls in ontology.classes()]
    
    def get_completions(self, document, complete_event):
        text = document.text
        for class_name in self.classes:
            if class_name.startswith(text):
                yield Completion(class_name, start_position=-len(text))

In [92]:
# Crea un oggetto Completer per le classi dell'ontologia
completer = OntologyCompleter(onto)

# Richiedi l'input dell'utente con autocompletamento
class_name = prompt("Inserisci il nome della classe (ad esempio, 'Action'): ", completer=completer, auto_suggest=AutoSuggestFromHistory())

# Cerca la classe corrispondente
item_class = onto.search(iri=f"*{class_name}")

if not item_class:
    print(f"Classe '{class_name}' non trovata nell'ontologia.")
else:
    item_class = item_class[0]  # Assumi che ci sia solo un risultato per il nome della classe

    # Trova tutti gli oggetti di tipo 'item_class'
    items = onto.search(is_a=item_class)

    print(f"Oggetti della classe '{item_class.name}':")
    for item in items:
        print(f"- {item.name}")

RuntimeError: asyncio.run() cannot be called from a running event loop

In [80]:
import owlready2
# Percorso al file dell'ontologia in formato Turtle
ontology_path = "data/Ontology/mtg_ontology.owl.tll"

# Crea un grafo RDF e carica il file Turtle usando rdflib
g = Graph()
g.parse(ontology_path, format="turtle")

# Salva il grafo RDF in un formato supportato da owlready2 (come RDF/XML)
converted_ontology_path = "converted_mtg_ontology.owl"
g.serialize(destination=converted_ontology_path, format="xml")

# Carica l'ontologia convertita usando owlready2
onto = get_ontology(converted_ontology_path).load()

obj_to_search = "actbion"
entity = onto.search(iri="*action")[0]
result = {}
# Determina il tipo dell'entità
if isinstance(entity, onto.Thing.__class__):
    print(f"'{entity.name}' è una classe.")
    result['sottoclassi'] = [subclass for subclass in entity.subclasses()]
    result['individui'] = [ind for ind in onto.individuals() if isinstance(ind, entity)]
    print(result)

elif isinstance(entity, onto.ObjectProperty):
    print(f"'{entity.name}' è una proprietà oggetto.")
    connected_entities = [range for range in entity.range]

elif isinstance(entity, onto.Thing):
    print(f"'{entity.name}' è un individuo.")
    connected_entities = [related for related in entity.related_individuals()]

else:
    print(f"'{entity.name}' è di un tipo sconosciuto.")
    connected_entities = []

# Stampa le entità collegate
if result:
    for type, items in result:
        print(f'{type} di {entity}:')
        for item in items:
            print(item.name)

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [49]:
# Stampa l'albero a partire dalla classe NamedThing
print("Albero dell'ontologia:")
thing_class = onto.Ability 
print(thing_class.name)
print_hierarchy(thing_class)

Albero dell'ontologia:
Ability
 ├ StaticAbility
 ├ ActivatedAbility
 └ TriggeredAbility


In [1]:
import requests
import time
import re

In [2]:
# Regex per estrarre l'ID del mazzo e il nome del mazzo
pattern = r'"name":"([^"]+)".*?"id":(\d+)'

# URL della richiesta GET
archidekt_url = 'https://archidekt.com/api/decks/'

#### Ottengo n mazzi
Tramite richiesta GET aggiungo al dizionario n mazzi unici e pubblici dall'API del sito https://archidekt.com, nel quale tutti gli utenti possono caricare e condividere i propri mazzi

In [3]:

all_decks = {}
page =1

while len(all_decks) < 200:
    # Effettuo la richiesta GET con questi parametri
    # pageSize: dimensione di ogni pagina di richiesta
    # formats: 3 è il formato Commander
    # page: pagina di richiesta corrente
    params = {'pageSize': 100, 'formats':3, 'page':page}
    time.sleep(0.1) # per non sovraccaricare la API
    response = requests.get(archidekt_url,params=params)
    
    # Verifica dello stato della risposta
    if response.status_code == 200:
        # Trova tutte le corrispondenze
        matches = re.findall(pattern, response.text)
        
        # Crea il dizionario con ID e nome del mazzo
        these_decks = {match[1]: {'name': match[0]} for match in matches}
        all_decks = {**these_decks, **all_decks}

        print(f'Mazzi fino a pagina {page}: {len(all_decks)}', end='\r')
        page+=1

    else:
        print(f'Errore {response.status_code}', end='\r')
        time.sleep(4)
    
print(f'{len(all_decks)} mazzi ottenuti dopo {page} pagine', end='\r')

KeyboardInterrupt: 

#### Forma di un oggetto Deck
Ad ogni mazzo (Deck) è associato un **id** unico su Archidekt, un **nome** assegnato dall'utente che l'ha caricato, e 100 e più carte al suo interno

i=0
for key,value in all_decks.items():
    i=i+1
    print(f'{i} - {key}: {value["name"]}')

## Carico tutte le carte
Ogni mazzo contiene 99 carte + il Commander (ci sono mazzi particolari in cui i commander sono 2).
Ogni carta è un dizionario coi seguenti campi:
```
{
    nome : stringa {
        uid : stringa
        editionCode : stringa da 3 caratteri
        manaValue : dizionario dei costi (interi) nei diversi colori
        manaProduction : dizionario del mana producibile dalla carta
        power : intero, potenza della creatura
        toughness : intero, costituzione della creatura
        superTypes : stringa, una tra Legendary, Basic o vuota 
        types : dizionario di stringhe, comprende Artefatto, Creatura, eccetera
        subTypes : dizionario di stringhe
        salt : decimale, numero assegnato da archidekt che indica quanto è detestata dagli avversari
        text : stringa, descrizione delle abilità della carta
        rarity : stringa, tra common, uncommon, rare, mythic
        defaultCategory : stringa, indica l'utilizzo più comune di una carta nel gioco    
    }
}

In [5]:
import json

global all_cards
all_cards = {}
num = 1

class Card:
    def __init__(self, uid, name, card_type, mana_cost, text, supertype, type, subytype, rarity):
        """
        Inizializza una nuova carta MTG.
        
        :param name: Nome della carta
        :param card_type: Tipo della carta (es. "Creature", "Sorcery", "Instant")
        :param mana_cost: Costo di mana della carta (es. "{2}{G}")
        :param text: Testo della carta (abilità, effetti, ecc.)
        :param power: Forza della carta (solo per le creature)
        :param toughness: Costituzione della carta (solo per le creature)
        :param set_name: Nome del set a cui appartiene la carta (opzionale)
        """
        self.name = name
        self.card_type = card_type
        self.mana_cost = mana_cost
        self.text = text
        self.power = power
        self.toughness = toughness
        self.set_name = set_name

def addCard(card):
    global all_cards
    {k: v for k, v in card.items() if v}

    all_cards ={
        **all_cards, 
        **{card['card']['oracleCard']['name']  : {
            "uid": card['card']['uid'],
            "colors": card['card']['oracleCard']['colors'],
            "colorIdentity": card['card']['oracleCard']['colorIdentity'],
            "manaValue": card['card']['oracleCard']['manaCost'],
            "manaProduction": card['card']['oracleCard']['manaProduction'],
            #"keywords": card['card']['keywords'],
            "power": card['card']['oracleCard']['power'],
            "toughness": card['card']['oracleCard']['toughness'],
            "superTypes": card['card']['oracleCard']['superTypes'],
            "types": card['card']['oracleCard']['types'],
            "subTypes": card['card']['oracleCard']['subTypes'],
            "salt": card['card']['oracleCard']['salt'],
            "text": card['card']['oracleCard']['text'],
            "rarity": card['card']['rarity'],
            "defaultCategory": card['card']['oracleCard']['defaultCategory']
        }}
    }


In [6]:

decks_to_delete = []
all_cards = []
for deck_id in all_decks.keys():
    time.sleep(0.15) # per non sovraccareicare la API
    response = requests.get(archidekt_url + deck_id +"/")
    
    # Verifica dello stato della risposta
    if response.status_code == 200:
        num+=1
        all_decks[deck_id]['cards'] = []
        all_decks[deck_id]['commanders'] = []
        
        deck = json.loads(response.text)
        if len(deck['cards'])> 93 and len(deck['cards'])<110:
            decks_to_delete.append(deck_id)
            continue

        for card in deck['cards']:
            # Escludo le carte non principali
            if card['categories'] and "Maybeboard" in card['categories']:
                continue          

            # Assegno i Commander al mazzo
            if card['categories'] and "Commander" in card['categories']:
                all_decks[deck_id]['commanders'].append(card['card']['uid'])
                continue
            
            try:
                # Popolo il dizionario
                addCard(card)
            except Exception as e:
                print(e)
                print(card)
            all_decks[deck_id]['cards'].append(card['card']['oracleCard']['name'])
            
        print(f'Carte uniche finora: {len(all_cards)}.     Mazzo {num} di {len(all_decks)}', end='\r')
            
    elif not response.status_code in [400, 404]:
        time.sleep(2)

for deck_id in decks_to_delete:
    del all_decks[deck_id]

print(f'\nMancano {len(all_decks)-num} mazzi all\'appello')



Carte uniche finora: 8003.     Mazzo 413 di 449
Mancano -49 mazzi all'appello


### DUDADADA

In [18]:
import pickle
with open("all_cards.pickle", "wb") as handle:
    pickle.dump(all_cards, handle)

In [19]:
def getCommander(deck_id):
    if len(all_decks[deck_id]['commanders']) == 2:
        return  all_decks[deck_id]['commanders'][0], all_decks[deck_id]['commanders'][1]
    return all_decks[deck_id]['commanders'][0]


# Proviamo ad Ontologizzare

In [92]:
from linkml_runtime.loaders import yaml_loader
from mtg_ontology.helpers import instance_to_graph 
from rdflib import Graph, Literal, RDF, URIRef, Namespace

# Specifica il percorso del tuo file .owl.tll
onto = "mtg_ontology.owl.tll"

# Crea un grafo RDF
g = Graph()

# Carica il file Turtle nel grafo RDF
g.parse(onto, format="turtle")

# Namespace per l'ontologia
Scryfall = Namespace("https://api.scryfall.com/cards/")
MTGO = Namespace("https://cmdoret.net/mtg_ontology/")
MTGFandom = Namespace("https://mtg.fandom.com/wiki/")
EDHREC = Namespace("https://edhrec.com/articles/new-edhrec-feature-salt/")

# Aggiungi il namespace al grafo
g.bind("scryfall", Scryfall)
g.bind("mtg", MTG)

# Funzione per creare URI per le istanze
def create_card_uri(card_uid):
    #return URIRef(Scryfall + card_uid)
    return URIRef(card_uid.replace(" ", "_"))

In [78]:
for s in g.subjects():
    print(f"soggetto: {s}")
    for p, o in g.predicate_objects(s):    
        print(f'oggetto:{o}, predicato:{p}')
    print("------------")


soggetto: https://cmdoret.net/mtg_ontology/
oggetto:http://www.w3.org/2002/07/owl#Ontology, predicato:http://www.w3.org/1999/02/22-rdf-syntax-ns#type
oggetto:mtgo, predicato:http://www.w3.org/2000/01/rdf-schema#label
oggetto:https://cmdoret.net/mtg_ontology/AbilityCollection, predicato:http://purl.obolibrary.org/obo/IAO_0000700
oggetto:https://cmdoret.net/mtg_ontology/CardCollection, predicato:http://purl.obolibrary.org/obo/IAO_0000700
oggetto:https://cmdoret.net/mtg_ontology/NamedThing, predicato:http://purl.obolibrary.org/obo/IAO_0000700
oggetto:https://cmdoret.net/mtg_ontology/Thing, predicato:http://purl.obolibrary.org/obo/IAO_0000700
oggetto:GNU GPL v3.0, predicato:http://purl.org/dc/terms/license
oggetto:Magic The Gathering Ontology., predicato:http://purl.org/dc/terms/title
oggetto:https://cmdoret.github.io/mtg_ontology/, predicato:http://www.w3.org/2000/01/rdf-schema#seeAlso
oggetto:2023-03-20T16:26:53, predicato:https://w3id.org/linkml/generation_date
oggetto:1.7.0, predicato:

In [86]:

for s in g.subjects(object=MTGO.AbilityKeyword):
    print(f"{s}")


https://mtg.fandom.com/wiki/deathtouch
https://mtg.fandom.com/wiki/defender
https://mtg.fandom.com/wiki/double_strike
https://mtg.fandom.com/wiki/enchant
https://mtg.fandom.com/wiki/equip
https://mtg.fandom.com/wiki/first_strike
https://mtg.fandom.com/wiki/flash
https://mtg.fandom.com/wiki/flying
https://mtg.fandom.com/wiki/haste
https://mtg.fandom.com/wiki/hexproof
https://mtg.fandom.com/wiki/indestructible
https://mtg.fandom.com/wiki/lifelink
https://mtg.fandom.com/wiki/menace
https://mtg.fandom.com/wiki/protection
https://mtg.fandom.com/wiki/reach
https://mtg.fandom.com/wiki/trample
https://mtg.fandom.com/wiki/vigilance
https://mtg.fandom.com/wiki/ward


In [95]:
for p, o in g.subject_objects(predicate=MTGFandom.defender):
    print(f"s:{p}              o:{o}")

for p, o in g.predicate_objects(subject=MTGFandom.defender):
    print(f"p:{p}              o:{o}")

for p, o in g.subject_predicates(object=MTGFandom.defender):
    print(f"s:{p}              p:{o}")

p:http://www.w3.org/1999/02/22-rdf-syntax-ns#type              o:http://www.w3.org/2002/07/owl#Class
p:http://www.w3.org/1999/02/22-rdf-syntax-ns#type              o:https://cmdoret.net/mtg_ontology/AbilityKeyword
p:http://www.w3.org/2000/01/rdf-schema#label              o:defender
s:n8e689804af64438aa9572450ff8bdd4ab178              p:http://www.w3.org/1999/02/22-rdf-syntax-ns#first
s:https://cmdoret.net/mtg_ontology/AbilityKeyword              p:https://w3id.org/linkml/permissible_values


In [99]:
for name, card in all_cards.items():
    try:
        card_uri = create_card_uri(name)
        #card_uri = create_card_uri(card['uid'])
        g.add((card_uri, RDF.type, MTG.Card))
        #g.add((card_uri, MTG.NamedThing, Literal(name)))
        g.add((card_uri, MTGO.color,                 Literal(card['colors'])))
        g.add((card_uri, MTGO.mana_cost,             Literal(card['manaValue'])))
        #g.add((card_uri, EDHREC.Just_a_Pinch,      Literal(card['salt'])))
        g.add((card_uri, MTGO.text,                  Literal(card['text'])))
        g.add((card_uri, MTGO.rarity,                Literal(card['rarity'])))
        g.add((card_uri, MTGO.hasDefaultCategory,    Literal(card['defaultCategory'])))#Rinnova categoria 
        g.add((card_uri, Scryfall.uid,              Literal(card['uid'])))
        g.add((card_uri, MTGO.card_type,             Literal(card["types"])))
        if card['subTypes']:
            g.add((card_uri, MTGO.card_subtype,      Literal(card['subTypes'])))
        if card['superTypes']:
            g.add((card_uri, MTGO.card_supertype,    Literal(card['superTypes'])))
        if card['manaProduction']:
            g.add((card_uri, MTGO.mana_production,   Literal(card['manaProduction'])))
        if card['power']:
            g.add((card_uri, MTGO.power,             Literal(card['power'])))
        if card['toughness']:
            g.add((card_uri, MTGO.toughness,         Literal(card['toughness'])))
    except Exception as e:
        print(e)
        continue

# Salva il grafo popolato
g.serialize("populated_mtg_ontology.yaml", format="yaml")


<Graph identifier=N5129041d7e934ef5bccbc52d3abb8bd3 (<class 'rdflib.graph.Graph'>)>

In [98]:
for p, o in g.subject_objects(predicate=MTGFandom.defender):
    print(f"s:{p}              o:{o}")

for p, o in g.predicate_objects(subject=MTGFandom.defender):
    print(f"p:{p}              o:{o}")

for p, o in g.subject_predicates(object=MTGFandom.defender):
    print(f"s:{p}              p:{o}")

p:http://www.w3.org/1999/02/22-rdf-syntax-ns#type              o:http://www.w3.org/2002/07/owl#Class
p:http://www.w3.org/1999/02/22-rdf-syntax-ns#type              o:https://cmdoret.net/mtg_ontology/AbilityKeyword
p:http://www.w3.org/2000/01/rdf-schema#label              o:defender
s:n8e689804af64438aa9572450ff8bdd4ab178              p:http://www.w3.org/1999/02/22-rdf-syntax-ns#first
s:https://cmdoret.net/mtg_ontology/AbilityKeyword              p:https://w3id.org/linkml/permissible_values


In [102]:
import random
from typing import get_args

s = list(g.subjects())[random.randint(1,len(g))-1]
# print(f"""
# {s} è una {MTG.Card}
# dai seguenti colori: {[c for c in g.objects(subject=s,predicate=MTG.color)]}
# """)

for p, o in g.predicate_objects(s):    
   print(f'{s} is a {o}, {p}')
        

Barrage_Tyrant is a https://cmdoret.net/mtg_ontology/Card, http://www.w3.org/1999/02/22-rdf-syntax-ns#type
Barrage_Tyrant is a [], https://cmdoret.net/mtg_ontology/color
Barrage_Tyrant is a {4}{R}, https://cmdoret.net/mtg_ontology/mana_cost
Barrage_Tyrant is a Devoid (This card has no color.)
{2}{R}, Sacrifice another colorless creature: Barrage Tyrant deals damage equal to the sacrificed creature's power to any target., https://cmdoret.net/mtg_ontology/text
Barrage_Tyrant is a rare, https://cmdoret.net/mtg_ontology/rarity
Barrage_Tyrant is a Removal, https://cmdoret.net/mtg_ontology/hasDefaultCategory
Barrage_Tyrant is a 08a83120-c0c5-4277-acae-6ecb6f6bf3e1, https://api.scryfall.com/cards/uid
Barrage_Tyrant is a ['Creature'], https://cmdoret.net/mtg_ontology/card_type
Barrage_Tyrant is a ['Eldrazi'], https://cmdoret.net/mtg_ontology/card_subtype
Barrage_Tyrant is a {'W': None, 'U': None, 'B': None, 'R': None, 'G': None, 'C': None}, https://cmdoret.net/mtg_ontology/mana_production
Barr

In [ ]:
from owlready2 import *

# Carica l'ontologia popolata
populated_onto = get_ontology("populated_mtg_ontology.yaml").load()

# Esegui il ragionamento
with populated_onto:
    sync_reasoner(keep_tmp_file=True)

print("lalalla")
print(populated_onto)

# Estrai le nuove conoscenze inferite
for card in populated_onto.Card.instances():
    print(card)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/axten/.local/lib/python3.10/site-packages/owlready2/hermit:/home/axten/.local/lib/python3.10/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmppk9hcld3


lalalla
get_ontology("https://cmdoret.net/mtg_ontology/")


* Owlready2 * HermiT took 1.868290662765503 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
